In [ ]:
import os
import pandas as pd
import numpy as np

In [ ]:
csv = "combined_data_MP_NE_dT_Coord_cC.csv"
df = pd.read_csv(csv)
df

In [ ]:
df.isna()

# calculate and add quarter

In [ ]:
df['quarter'] = (df['month'] - 1) // 3 + 1
df

In [ ]:
df["quarter"].unique()

# add number of day

In [ ]:
weekday_mapping = {'Monday': 0, 'Tuesday': 1, 'Wednesday': 2, 'Thursday': 3, 'Friday': 4, 'Saturday': 5, 'Sunday/Holiday': 6}
df['weekday_number'] = df['weekday'].map(weekday_mapping)
df

In [ ]:
df['weekday_number'].unique()

# add isWeekend

In [ ]:
# Create the new column 'isWeekend' using numpy.where()
df['isWeekend'] = np.where(df['weekday'].isin(['Saturday', 'Sunday/Holiday']), True, False)
df

In [ ]:
contains_nan = df.isnull().any()
contains_nan

In [ ]:
df.to_csv(csv.split(sep='.')[0] + '_fE' + '.csv', index=False)

In [ ]:
# Assign quarter based on month
df['quarter'] = (df['month'] - 1) // 3 + 1

# Add number of day in the week
weekday_mapping = {'Monday': 0, 'Tuesday': 1, 'Wednesday': 2,
                   'Thursday': 3, 'Friday': 4, 'Saturday': 5,
                   'Sunday/Holiday': 6}
df['weekday_number'] = df['weekday'].map(weekday_mapping)

# Add isWeekend flag
df['isWeekend'] = np.where(df['weekday'].isin(
    ['Saturday', 'Sunday/Holiday']), True, False)

# Add OSM data as features (like counts for schools etc.)

In [ ]:
import pandas as pd

In [ ]:
csv = "combined_data_MP_NE_mappedUCOtoMTC_Coord_dT_fE_mergedClusterInside_cC_start.csv"
df = pd.read_csv(csv)
df

In [ ]:
df.count_corrected.describe()

In [ ]:
csv_osm = "osm_data_cleaned.csv"
df_osm = pd.read_csv(csv_osm)
df_osm

In [ ]:
df_osm.clusterID.unique()

In [ ]:
xlsx = "mappingInsideStudyArea.xlsx"
df_xlsx = pd.read_excel(xlsx)
df_xlsx

In [ ]:
# Create a mapping dictionary from df_api
mapping_dict_inside = df_xlsx.set_index('search_id')['id'].to_dict()
mapping_dict_inside

In [ ]:
# Apply the mapping to the column in df, keeping original value if no match
df_osm['clusterID'] = df_osm['clusterID'].apply(lambda x: mapping_dict_inside.get(x, x))
df_osm

In [ ]:
# Apply the mapping to the column in df, keeping original value if no match
df_osm['cluster'] = df_osm['cluster'].apply(lambda x: x.split(' (')[0])
df_osm

In [ ]:
df_osm.cluster.unique()

one way

In [ ]:
df_osm.drop(columns='cluster', inplace=True)
df_osm

another way

In [ ]:
df_osm = df_osm.groupby(['clusterID']).agg({
    'kindergartens_count': 'sum',
    'schools_count': 'sum',
    'sportsCentres_count': 'sum',
    'publicTransportStops_count': 'sum',
    'supermarkets_count': 'sum'
})
df_osm

In [ ]:
df_merged = df.merge(df_osm, how='left', left_on='startClusterID', right_on='clusterID')
df_merged.fillna(0, inplace=True)
df_merged

In [ ]:
df_merged.dtypes

In [ ]:
#df_merged.drop(columns='clusterID', inplace=True)
df_merged['kindergartens_count'] = df_merged['kindergartens_count'].astype(int)
df_merged['schools_count'] = df_merged['schools_count'].astype(int)
df_merged['sportsCentres_count'] = df_merged['sportsCentres_count'].astype(int)
df_merged['publicTransportStops_count'] = df_merged['publicTransportStops_count'].astype(int)
df_merged['supermarkets_count'] = df_merged['supermarkets_count'].astype(int)
df_merged

In [ ]:
df_merged.to_csv(csv.split(sep='.')[0] + '_OSM' + '.csv', index=False)

# Clip count & Distance metrics

In [ ]:
csv = "combined_data_MP_NE_mappedUCOtoMTC_Coord_dT_fE_mergedClusterInside_cC_start_OSM.csv"
df = pd.read_csv(csv)
df

In [ ]:
# Sort the data by 'year', 'quarter', and 'month'
df = df.sort_values(by=['year', 'quarter', 'month'])

# Add lag features for the last 3 months
df['count_lag1'] = df.groupby(['startClusterID'])['count_corrected'].shift(periods=1)
df['count_lag2'] = df.groupby(['startClusterID'])['count_corrected'].shift(periods=2)
df['count_lag3'] = df.groupby(['startClusterID'])['count_corrected'].shift(periods=3)

# Calculate the sum of 'count_corrected' of the previous month for each 'startClusterID'
df['sum_of_count_lag1'] = df.groupby(['startClusterID'])['count_corrected'].shift(periods=1).groupby(df['startClusterID']).cumsum()
df['sum_of_count_lag2'] = df.groupby(['startClusterID'])['count_corrected'].shift(periods=2).groupby(df['startClusterID']).cumsum()
df['sum_of_count_lag3'] = df.groupby(['startClusterID'])['count_corrected'].shift(periods=3).groupby(df['startClusterID']).cumsum()

# Calculate the metrics for 'sum_of_count_previous_month' for each 'startClusterID'
metrics = df.groupby(['startClusterID'])['sum_of_count_lag1'].agg(['std', 'median', ('25th_percentile', lambda x: x.quantile(0.25)), ('75th_percentile', lambda x: x.quantile(0.75))]).reset_index()

# Rename the columns to include the metric names
metrics.columns = ['startClusterID', 'std_of_previous_month', 'median_of_previous_month', 'quantile_25_of_previous_month', 'quantile_75_of_previous_month']

# Merge the metrics DataFrame with the original DataFrame based on 'startClusterID'
df = df.merge(metrics, on=['startClusterID'], how='left')

df

In [ ]:
# Sort the data by 'year', 'quarter', and 'month'
df = df.sort_values(by=['year', 'quarter', 'month'])

# Add lag features for the last 3 months
df['count_lag1'] = df.groupby(['startClusterID', 'endClusterID'])['count_corrected'].shift(periods=1)
df['count_lag2'] = df.groupby(['startClusterID', 'endClusterID'])['count_corrected'].shift(periods=2)
df['count_lag3'] = df.groupby(['startClusterID', 'endClusterID'])['count_corrected'].shift(periods=3)

# Calculate the sum of 'count_corrected' of the previous month for each 'startClusterID'
df['sum_of_count_lag1'] = df.groupby(['startClusterID', 'endClusterID'])['count_corrected'].shift(periods=1).groupby(df['startClusterID']).cumsum()
df['sum_of_count_lag2'] = df.groupby(['startClusterID', 'endClusterID'])['count_corrected'].shift(periods=2).groupby(df['startClusterID']).cumsum()
df['sum_of_count_lag3'] = df.groupby(['startClusterID', 'endClusterID'])['count_corrected'].shift(periods=3).groupby(df['startClusterID']).cumsum()


# Calculate the metrics for 'sum_of_count_previous_month' for each 'startClusterID'
metrics = df.groupby(['startClusterID', 'endClusterID'])['sum_of_count_lag1'].agg(['std', 'median', ('25th_percentile', lambda x: x.quantile(0.25)), ('75th_percentile', lambda x: x.quantile(0.75))]).reset_index()

# Rename the columns to include the metric names
metrics.columns = ['startClusterID', 'endClusterID', 'std_of_previous_month', 'median_of_previous_month', 'quantile_25_of_previous_month', 'quantile_75_of_previous_month']

# Merge the metrics DataFrame with the original DataFrame based on 'startClusterID'
df = df.merge(metrics, on=['startClusterID', 'endClusterID'], how='left')

df

In [ ]:
df.isna().sum()

In [ ]:
# Fill NaN values with 0 (for the first month of each startClusterID)
df.fillna(0, inplace=True)

In [ ]:
# Sort the data by 'year', 'quarter', and 'month'
df = df.sort_values(by=['year', 'quarter', 'month'])

# Add lag features of 'count_corrected' for the last 3 months
df['count_lag1'] = df.groupby(['startClusterID'])['count_corrected']\
    .shift(periods=1)
df['count_lag2'] = df.groupby(['startClusterID'])['count_corrected']\
    .shift(periods=2)
df['count_lag3'] = df.groupby(['startClusterID'])['count_corrected']\
    .shift(periods=3)

# Add lag features of the sum of 'count_corrected' for the last 3 months
df['sum_of_count_lag1'] = df.groupby(['startClusterID'])['count_corrected']\
    .shift(periods=1).groupby(df['startClusterID']).cumsum()
df['sum_of_count_lag2'] = df.groupby(['startClusterID'])['count_corrected']\
    .shift(periods=2).groupby(df['startClusterID']).cumsum()
df['sum_of_count_lag3'] = df.groupby(['startClusterID'])['count_corrected']\
    .shift(periods=3).groupby(df['startClusterID']).cumsum()

# Calculate the metrics for 'sum_of_count_previous_month' for each 'startClusterID'
metrics = df.groupby(['startClusterID'])['sum_of_count_lag1'].agg([
    'std', 'median', ('25th_percentile', lambda x: x.quantile(0.25)),
    ('75th_percentile', lambda x: x.quantile(0.75))]).reset_index()
# Rename the columns to include the metric names
metrics.columns = ['startClusterID', 'std_of_previous_month',
                   'median_of_previous_month', 'quantile_25_of_previous_month',
                   'quantile_75_of_previous_month']
# Merge the metrics DataFrame with the original DataFrame based on 'startClusterID'
df = df.merge(metrics, on=['startClusterID'], how='left')

# Fill NaN values with 0 (relevant for lag features)
df.fillna(0, inplace=True)

In [ ]:
# Sort the data by 'year', 'quarter', and 'month'
df = df.sort_values(by=['year', 'quarter', 'month'])

# Add lag features of 'count_corrected' for the last 3 months
df['count_lag1'] = df.groupby(['startClusterID', 'endClusterID'])['count_corrected']\
    .shift(periods=1)
df['count_lag2'] = df.groupby(['startClusterID', 'endClusterID'])['count_corrected']\
    .shift(periods=2)
df['count_lag3'] = df.groupby(['startClusterID', 'endClusterID'])['count_corrected']\
    .shift(periods=3)

# Calculate the sum of 'count_corrected' of the previous month for each 'startClusterID'
df['sum_of_count_lag1'] = df.groupby(['startClusterID', 'endClusterID'])['count_corrected']\
    .shift(periods=1).groupby(df['startClusterID']).cumsum()
df['sum_of_count_lag2'] = df.groupby(['startClusterID', 'endClusterID'])['count_corrected']\
    .shift(periods=2).groupby(df['startClusterID']).cumsum()
df['sum_of_count_lag3'] = df.groupby(['startClusterID', 'endClusterID'])['count_corrected']\
    .shift(periods=3).groupby(df['startClusterID']).cumsum()

# Calculate the metrics for 'sum_of_count_previous_month' for each 'startClusterID'
metrics = df.groupby(['startClusterID', 'endClusterID'])['sum_of_count_lag1'].agg([
    'std',
    'median',
    ('25th_percentile', lambda x: x.quantile(0.25)),
    ('75th_percentile', lambda x: x.quantile(0.75))
]).reset_index()
# Rename the columns to include the metric names
metrics.columns = ['startClusterID', 'endClusterID', 'std_of_previous_month',
                   'median_of_previous_month', 'quantile_25_of_previous_month',
                   'quantile_75_of_previous_month']
# Merge the metrics DataFrame with the original DataFrame based on 'startClusterID'
df = df.merge(metrics, on=['startClusterID', 'endClusterID'], how='left')

# Fill NaN values with 0 (relevant for lag features)
df.fillna(0, inplace=True)

In [ ]:
df

In [ ]:
df.to_csv(csv.split(sep='.')[0] + '_metrics' + '.csv', index=False)

random stuff

In [ ]:
import matplotlib.pyplot as plt
# Create a scatter plot
plt.figure(figsize=(10, 6))
plt.scatter(df['distance'], df['count_corrected'], alpha=0.5)

plt.xlabel('Distance', fontsize=14)
plt.ylabel('Count Corrected', fontsize=14)
plt.title('Relationship between Count Corrected and Distance', fontsize=16)

plt.show()

In [ ]:
df['distance'].describe()

In [ ]:
from scipy.stats import linregress

# Assuming 'df' is your DataFrame with the 'count_corrected' and 'count_of_previous_month' columns

# Linear Regression
x = df['distance']
y = df['count_corrected']
slope, intercept, r_value, p_value, std_err = linregress(x, y)

# Calculate R-squared
r_squared = r_value**2

# Correlation
correlation = x.corr(y)

print("Linear Regression:")
print("Slope:", slope)
print("Intercept:", intercept)
print("R-squared:", r_squared)

print("\nCorrelation:")
print("Correlation coefficient:", correlation)

In [ ]:
# Sort the data by 'year', 'quarter', and 'month'
df = df.sort_values(by=['year', 'quarter', 'month'])

# Add lag features for the last 3 months
df['count_lag1'] = df.groupby(['startClusterID'])['count_corrected'].shift(periods=1)
df['count_lag2'] = df.groupby(['startClusterID'])['count_corrected'].shift(periods=2)
df['count_lag3'] = df.groupby(['startClusterID'])['count_corrected'].shift(periods=3)

# Calculate the sum of 'count_corrected' of the previous month for each 'startClusterID'
df['sum_of_count_lag1'] = df.groupby(['startClusterID'])['count_corrected'].shift(periods=1).groupby(df['startClusterID']).cumsum()
df['sum_of_count_lag2'] = df.groupby(['startClusterID'])['count_corrected'].shift(periods=2).groupby(df['startClusterID']).cumsum()
df['sum_of_count_lag3'] = df.groupby(['startClusterID'])['count_corrected'].shift(periods=3).groupby(df['startClusterID']).cumsum()


# Calculate the metrics for 'sum_of_count_previous_month' for each 'startClusterID'
metrics = df.groupby(['startClusterID'])['sum_of_count_lag1'].agg(['std', 'median', ('25th_percentile', lambda x: x.quantile(0.25)), ('75th_percentile', lambda x: x.quantile(0.75))]).reset_index()

# Rename the columns to include the metric names
metrics.columns = ['startClusterID', 'std_of_previous_month', 'median_of_previous_month', 'quantile_25_of_previous_month', 'quantile_75_of_previous_month']

# Merge the metrics DataFrame with the original DataFrame based on 'startClusterID'
df = df.merge(metrics, on='startClusterID', how='left')

df

In [ ]:
df.isna().sum()

In [ ]:
# Fill NaN values with 0 (for the first month of each startClusterID)
df.fillna(0, inplace=True)

In [ ]:
df['count_lag1'].value_counts()

In [ ]:
import matplotlib.pyplot as plt
from scipy.stats import linregress

# Assuming 'df' is your DataFrame with the 'count_corrected' and 'count_of_previous_month' columns

# Linear Regression
x = df['count_corrected']
y = df['count_lag1']
slope, intercept, r_value, p_value, std_err = linregress(x, y)

# Calculate R-squared
r_squared = r_value**2

# Correlation
correlation = x.corr(y)

print("Linear Regression:")
print("Slope:", slope)
print("Intercept:", intercept)
print("R-squared:", r_squared)

print("\nCorrelation:")
print("Correlation coefficient:", correlation)

# Create a scatter plot
plt.figure(figsize=(10, 6))
plt.scatter(x, y, alpha=0.5)

plt.xlabel(f'{x.name}', fontsize=14)
plt.ylabel(f'{y.name}', fontsize=14)

plt.show()

In [ ]:
df_id = df.groupby(['startClusterID', 'startClusterName'])['count_corrected'].sum().reset_index()
df_id

In [ ]:
# Calculate the sum of count_corrected for each group
group_sums = df_id.groupby('startClusterID')['count_corrected'].sum()

# Sort the groups by sum in descending order
sorted_groups = group_sums.sort_values(ascending=True)

# Create a dictionary to map old startClusterID values to new ones
cluster_id_mapping = {old_id: new_id for new_id, old_id in enumerate(sorted_groups.index)}

# Apply the mapping to create a new column with updated startClusterID values
df_id['new_startClusterID'] = df_id['startClusterID'].map(cluster_id_mapping)

df_id

In [ ]:
# Replace the startClusterID values in the original df using the mapping
df['startClusterID'] = df['startClusterID'].map(cluster_id_mapping)
df

In [ ]:
df.drop(columns=['startClusterName', 'startClusterZip', 'count_lag2', 'count_lag3','sum_of_count_lag1', 'sum_of_count_lag2', 'sum_of_count_lag3'], inplace=True)

In [ ]:
df.to_csv(csv.split(sep='.')[0] + '_metrics' + '.csv', index=False)

# unrelated - combine routes

In [ ]:
csv = "combined_data_MP_NE_with_coordinatessumCountPerRoutes.csv"
df = pd.read_csv(csv)
df

In [ ]:
import numpy as np
"""df['StartLon'] = df['StartLon'].astype(str)
df['StartLat'] = df['StartLat'].astype(str)
df['EndLon'] = df['EndLon'].astype(str)
df['EndLat'] = df['EndLat'].astype(str)"""

# Sort the values in each row to treat combinations as equivalent
df[['StartName', 'EndName']] = pd.DataFrame(np.sort(df[['StartName', 'EndName']], axis=1), index=df.index)

# Group by the sorted combinations and aggregate other columns
df_grouped = df.groupby(['StartName', 'EndName'], as_index=False).agg({
    'Count': 'sum'
})
df_grouped

In [ ]:
df.Count.sum()

In [ ]:
df_grouped.Count.sum()

In [ ]:
df_grouped.to_csv(csv.split('.')[0] + '_sumPerRoutesBi.csv', index=False)

In [ ]:
csv = "combined_data_MP_NE_with_coordinatessumCountPerRoutes_sumPerRoutesBi.csv"
df = pd.read_csv(csv)
df

In [ ]:
df_start = df.groupby(['StartName'], as_index=False).agg({
    'Count': 'sum'
})
df_start

In [ ]:
df_end = df.groupby(['EndName'], as_index=False).agg({
    'Count': 'sum'
})
df_end

In [ ]:
df_sum = pd.merge(df_start, df_end, how='outer', left_on='StartName', right_on='EndName')
df_sum

In [ ]:
df_sum.fillna(0, inplace=True)
df_sum

In [ ]:
df_sum['Count'] = df_sum['Count_x'] + df_sum['Count_y']
df_sum

In [ ]:
df_sum.drop(columns=['Count_x', 'Count_y'], inplace=True)
df_sum

In [ ]:
df_sum.loc[df_sum['StartName'] == 0, 'StartName'] = df_sum['EndName']
df_sum

In [ ]:
df_sum = df_sum.groupby('StartName')['Count'].sum().reset_index()
df_sum

In [ ]:
df_sum.Count.describe()

In [ ]:
csv_shapes = "desd-4-landkreis-hildesheim-1663168323029-shapes.csv"
df_shapes = pd.read_csv(csv_shapes)
df_shapes

In [ ]:
df_sum.rename(columns={'StartName': 'name'}, inplace=True)
df_sum

In [ ]:
df_sum = df_sum.merge(df_shapes, on='name', how='left')
df_sum

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Fit and transform the 'Count' column
df_sum['Count_normalized'] = scaler.fit_transform(df_sum[['Count']])
df_sum

In [ ]:
df_sum.to_csv(csv.split('.')[0] + '_sumPerCluster.csv', index=False)